In [ ]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
# Main Window class
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main layout
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

 # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        # Save the project to the database
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        # Load the project from the database
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()
        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        # Display a status message indicating the active project
        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
# Main Window class
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main layout
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

 # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        # Save the project to the database
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
# Main Window class
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main layout
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

 # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
    # Save the project to the database
    labels = [label.to_dict() for label in self.right_panel.labels]  # Collect all labels
    save_project_to_db(project_name, project_location, labels)  # Save to DB
    
    
    def load_project(self, project_id):
        # Load the project from the database
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()
        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        # Display a status message indicating the active project
        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

SystemExit: 0

C:\Users\Ndivheni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
pip install mysql-connector-python


Note: you may need to restart the kernel to use updated packages.


In [4]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
# Main Window class
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main layout
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

 # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        # Save the project to the database
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
# Main Window class
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main layout
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

 # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
    """Save the project to the database."""
    labels = [label.to_dict() for label in self.right_panel.labels]  # Collect all labels
    save_project_to_db(project_name, project_location, labels)  # Save to DB
    
    
    def load_project(self, project_id):
        # Load the project from the database
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()
        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        # Display a status message indicating the active project
        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

IndentationError: expected an indented block (1777096511.py, line 651)

In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
# Main Window class
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()

        # Main layout
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

 # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        # Save the project to the database
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        # Load the project from the database
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()
        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        # Display a status message indicating the active project
        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

SystemExit: 0

C:\Users\Ndivheni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()  # Corrected line

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)  # Corrected line
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        # Generate tooltip text from properties
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        # Convert the label to a dictionary format for saving
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        # Create a MovableLabel from a dictionary
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)  # Set a fixed size for the arrow widget
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        
        # Draw the arrow line
        painter.drawLine(self.start_point, self.end_point)

        # Draw the arrowhead
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10  # Length of the arrowhead
        arrow_angle = 30  # Angle of the arrowhead

        # Calculate the angle of the arrow
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        # Calculate the points for the arrowhead
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )

        # Draw the arrowhead
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
        
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        
        # Main layout setup
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        # Menu bar with File and Exit options
        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        # Add "New Project" action to the File menu
        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        # Add "Exit" action to the File menu
        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        # Central widget for layout
        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        # Left panel (legend)
        left_panel = self.create_legend_panel()

        # Right panel (drop area)
        self.right_panel = self.create_drop_panel()

        # Splitter between left and right panels
        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])  # Adjust size ratios of left and right panels

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        # Status bar
        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)
        
        
    def create_legend_panel(self):
        # Create the left panel (legend) with the elements as draggable labels
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        # Create the right panel (drop area, bigger)
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        # Logic to pop up dialog for entering project name and location
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()  # Clear the workspace for a new project
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def clear_workspace(self):
        # Clear the right panel for a fresh start
        self.right_panel.clear()

    def add_current_project(self, project_name):
        # Create or get the "Current Project" menu
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        # Add project to the "Current Project" menu
        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        # Save the project to the database
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        # Load the project from the database
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()  # Clear the workspace before loading

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()  # Show the label in the workspace
            self.right_panel.labels.append(label)  # Keep track of labels

        # Display a status message indicating the active project
        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

# Custom dialog for New Project
class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        # Layout
        layout = QVBoxLayout()

        # Project name input
        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        # Project location input
        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        # Browse button for location
        browse_button = QPushButton("", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        # Save, Delete, and Preview buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)  # Save action
        button_layout.addWidget(save_button)

        delete_button = QPushButton("", self)
        delete_button.clicked.connect(self.delete_project)  # Delete action
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("", self)
        preview_button.clicked.connect(self.preview_project)  # Preview action
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        # Open file dialog to select project location
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        # Return project name and location entered by the user
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        # Placeholder for delete logic
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
 # Placeholder for preview logic
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []  # Keep track of all dropped labels
        self.arrows = []  # Keep track of all drawn arrows

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        # Create a MovableLabel and position it where dropped
        label = MovableLabel(text, self)
        label.move(event.pos())  # Move to the drop position
        label.show()
        self.labels.append(label)

        # Check if the dropped element is a connector
        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                # Create an Arrow widget and add it to the drop area
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)  # Default length of 50
                arrow.show()
                self.arrows.append(arrow)

        # Show properties popup to enter element properties
        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        # Clear the right panel (drop area)
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow .deleteLater()
        self.arrows.clear()

# Custom dialog for entering element properties
class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        # Layout
        layout = QVBoxLayout()

        # Form fields for properties
        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        # Save and Cancel buttons
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        # Save the properties entered by the user
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())  

SystemExit: 0

C:\Users\Ndivheni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.add ```python
            widget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event ```python
):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax (2332762713.py, line 268)

In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event .button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

C:\Users\Ndivheni\AppData\Local\Temp\ipykernel_8268\1298084360.py:182: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p1 = QPoint(
C:\Users\Ndivheni\AppData\Local\Temp\ipykernel_8268\1298084360.py:186: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p2 = QPoint(


SystemExit: 0

C:\Users\Ndivheni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from PyQt5.QtWidgets import QApplication, QWidget, QListWidget, QHBoxLayout, QListWidgetItem, QSizePolicy, QMainWindow, QAction, QMenu, QPushButton, QToolBar
from PyQt5.QtGui import QIcon, QPainter, QPen, QPolygonF, QColor, QPainterPath
from PyQt5.QtCore import Qt, QPoint, QPointF
import sys
import math

class Window(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD")
        self.setGeometry(300, 350, 900, 600)

        # Menu bar
        menubar = self.menuBar()
        fileMenu = menubar.addMenu('&File')

        exitAction = QAction(QIcon('exit.png'), '&Exit', self)
        exitAction.setShortcut('Ctrl+Q')
        exitAction.setStatusTip('Exit application')
        exitAction.triggered.connect(self.close)
        fileMenu.addAction(exitAction)

        viewMenu = menubar.addMenu('&View')
        
        # Add Hide Arrows action to View menu
        hide_arrows_action = QAction("Hide Arrows", self)
        hide_arrows_action.triggered.connect(self.hide_all_right_panel)
        viewMenu.addAction(hide_arrows_action)
        
        # Toolbar
        toolbar = QToolBar("Tools")
        self.addToolBar(toolbar)

        # Toggle Visibility button on toolbar
        toggle_visibility_action = QAction("", self)
        toggle_visibility_action.triggered.connect(self.toggle_visibility)
        toolbar.addAction(toggle_visibility_action)

        # View Button on toolbar
        view_button = QPushButton("View")
        toolbar.addWidget(view_button)
        view_menu = QMenu()
        view_action = QAction("Label", self)
        view_action.triggered.connect(lambda: self.set_view_mode("Label"))
        view_menu.addAction(view_action)
        hide_action = QAction("Hide", self)
        hide_action.triggered.connect(lambda: self.toggle_labels("hide"))
        view_menu.addAction(hide_action)
        show_action = QAction("Show", self)
        show_action.triggered.connect(lambda: self.toggle_labels("show"))
        view_menu.addAction(show_action)

        view_button.setMenu(view_menu)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.initUI()
        self.connections = []  # List to store connections between items
        self.start_item = None  # Store the item where the click started
        self.drawing_arrow = False  # Flag to indicate arrow drawing mode
        self.arrow_start_pos = None  # Store the starting position of the arrow
        self.arrow_counter = {
            "runoff": 1,
            "raw water abstraction": 1,
            "treated water": 1,
            "greywater reuse": 1,
            "wastewater": 1,
            "streamflow": 1,
            "treated wastewater": 1,
            "feedback loop": 1
        }

    def initUI(self):
        self.myListWidget1 = QListWidget()
        self.myListWidget2 = ArrowDrawingWidget()  # Use custom widget for arrow drawing
        self.myListWidget2.setViewMode(QListWidget.IconMode)
        self.myListWidget1.setAcceptDrops(True)
        self.myListWidget1.setDragEnabled(True)
        self.myListWidget2.setAcceptDrops(True)
        self.myListWidget2.setDragEnabled(True)
        self.myListWidget1.setSizePolicy(QSizePolicy.Maximum, QSizePolicy.Preferred)
        self.myListWidget2.setSizePolicy(QSizePolicy.Expanding, QSizePolicy.Preferred)

        hboxlayout = QHBoxLayout(self.central_widget)
        hboxlayout.addWidget(self.myListWidget1)
        hboxlayout.addWidget(self.myListWidget2, stretch=2)

        items1 = [
            ("________________________________", ""),
            ("***     NODES       ***", ""),
            ("________________________________", ""),
            ("dam", "blue triangle.png"),
            ("reservoir", "icon2.jpg"),
            ("pump station", "icon3.png"),
            ("mine", "icon5.png"),
            ("rainwater harvesting", "harvest1.jpg"),
            ("farm block ", "farm block.png"),
            ("stormwater ", "stormwater.png"),
            ("groundwater harvesting", "groundwater harvesting.jpg"),
            ("groundwater source", "red diamond.png"),
            ("ecosystem infrastructure", "ecosystem.jpg"),
            ("mine dam", "brown triangle.png"),
            ("water treatment plant", "blue circle.jpg"),
            ("wastewater treatment", "red circle.png"),
            ("junction", "screenshot14.png"),
            ("subarea", "rectangle1.png"),
            ("water supply area", "pentagon.jpg"),
            ("settlement", "red star.png"),
            ("power station", "power station.png"),
            ("industry", "industry.png"),
            ("agriculture", "agriculture.jpg"),
            ("________________________________", ""),
            ("***       DATA      ***", ""),
            ("________________________________", ""),
            ("water quality monitoring point ", "screenshot1.png"),
            ("streamflow gauging station", "screenshot2.png"),
            ("water meter", "screenshot13.png"),
            ("________________________________", ""),
            ("***       CONNECTORS       ***", ""),
            ("________________________________", ""),
            ("runoff", "runoff1.png"),
            ("raw water abstraction", "raw water1.png"),
            ("treated water", "treated1.png"),
            ("greywater reuse", "grey.png"),
            ("wastewater", "wastee.png"),
            ("streamflow", "streamflow1.png"),
            ("treated wastewater", "treatedwaste1.png"),
            ("feedback loop", "feedback.png"),
        ]

        for text, icon_path in items1:
            item = QListWidgetItem(QIcon(icon_path), text)
            item.setData(Qt.UserRole, text)  # Store original text
            self.myListWidget1.addItem(item)
            # Disable drag for specific items
            if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
                item.setFlags(item.flags() & ~Qt.ItemIsDragEnabled)

        self.myListWidget1.itemClicked.connect(self.item_clicked)
        self.show()

    def item_clicked(self, item):
        if item.text() in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            self.drawing_arrow = True
            self.myListWidget2.start_drawing_arrow(item.text())

    def toggle_visibility(self):
        # Toggle visibility of items in myListWidget2 only
        for i in range(self.myListWidget2.count()):
            item = self.myListWidget2.item(i)
            item.setHidden(not item.isHidden())

    def hide_all_right_panel(self):
        # Hide both arrows and items in the right panel (ArrowDrawingWidget)
        self.myListWidget2.hide_all_arrows()

    def set_view_mode(self, mode):
        if mode == "Label":
            for i in range(self.myListWidget2.count()):
                item = self.myListWidget2.item(i)
                item.setText("")  # Clear text for all items
            self.myListWidget2.setViewMode(QListWidget.IconMode)  # Set view mode to IconMode
        else:
            self.myListWidget2.setViewMode(QListWidget.ListMode)  # Set view mode to ListMode

    def toggle_labels(self, action):
        for i in range(self.myListWidget2.count()):
            item = self.myListWidget2.item(i)
            if action == "hide":
                item.setText("")  # Hide text
            elif action == "show":
                item.setText(item.data(Qt.UserRole))  # Restore text

class ArrowDrawingWidget(QListWidget):
    def __init__(self):
        super().__init__()
        self.drawing_arrow = False
        self.arrow_start_pos = None
        self.connections = []
        self.arrow_type = None
        self.selected_connection = None
        self.dragging_start = False
        self.dragging_end = False
        self.arrow_counter = {
            "runoff": 1,
            "raw water abstraction": 1,
            "treated water": 1,
            "greywater reuse": 1,
            "wastewater": 1,
            "streamflow": 1,
            "treated wastewater": 1,
            "feedback loop": 1
        }

    def start_drawing_arrow(self, arrow_type):
        self.drawing_arrow = True
        self.arrow_type = arrow_type

    def mousePressEvent(self, event):
        if self.drawing_arrow:
            self.arrow_start_pos = event.pos()
        else:
            for connection in self.connections:
                start, end, _ = connection
                if self.is_near(event.pos(), start):
                    self.selected_connection = connection
                    self.dragging_start = True
                    break
                elif self.is_near(event.pos(), end):
                    self.selected_connection = connection
                    self.dragging_end = True
                    break
            super().mousePressEvent(event)

    def mouseMoveEvent(self, event):
        if self.dragging_start and self.selected_connection:
            start, end, arrow_type = self.selected_connection
            self.connections.remove(self.selected_connection)
            self.connections.append((event.pos(), end, arrow_type))
            self.selected_connection = (event.pos(), end, arrow_type)
            self.viewport().update()
        elif self.dragging_end and self.selected_connection:
            start, end, arrow_type = self.selected_connection
            self.connections.remove(self.selected_connection)
            self.connections.append((start, event.pos(), arrow_type))
            self.selected_connection = (start, event.pos(), arrow_type)
            self.viewport().update()
        else:
            super().mouseMoveEvent(event)

    def mouseReleaseEvent(self, event):
        if self.drawing_arrow and self.arrow_start_pos:
            if not self.is_near(self.arrow_start_pos, event.pos()):
                # Create a new connection with a unique label
                label = f"{self.arrow_type} {self.arrow_counter[self.arrow_type]}"
                self.connections.append((self.arrow_start_pos, event.pos(), self.arrow_type))
                self.arrow_counter[self.arrow_type] += 1
                self.drawing_arrow = False
                self.arrow_start_pos = None
                self.viewport().update()  # Trigger a repaint to show the arrow
            else:
                self.drawing_arrow = False
                self.arrow_start_pos = None
                self.viewport().update()
        elif self.dragging_start or self.dragging_end:
            self.dragging_start = False
            self.dragging_end = False
            self.selected_connection = None
            self.viewport().update()
        else:
            super().mouseReleaseEvent(event)

    def paintEvent(self, event):
        super().paintEvent(event)
        painter = QPainter(self.viewport())
        for start, end, arrow_type in self.connections:
            if arrow_type == "runoff":
                pen = QPen(QColor(165, 42, 42), 2, Qt.DashLine)  # Brown dashed line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "raw water abstraction":
                pen = QPen(QColor(165, 42, 42), 2, Qt.SolidLine)  # Brown solid line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "treated water":
                pen = QPen(QColor(173, 216, 230), 2, Qt.SolidLine)  # Light blue solid line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "greywater reuse":
                pen = QPen(Qt.gray, 2, Qt.DashLine)  # Grey dashed line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "wastewater":
                pen = QPen(Qt.green, 2, Qt.SolidLine)  # Green solid line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "streamflow":
                pen = QPen(QColor(0, 0, 139), 2, Qt.SolidLine)  # Dark blue solid line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "treated wastewater":
                pen = QPen(Qt.green, 2, Qt.DashLine)  # Green dashed line
                self.draw_text_in_middle(painter, start, end, "")
            elif arrow_type == "feedback loop":
                # Draw a curved arrow for feedback loop
                path = QPainterPath()
                path.moveTo(start)
                control_point = QPointF((start.x() + end.x()) / 2, start.y() - 50)
                path.quadTo(control_point, end)
                painter.setPen(QPen(Qt.black, 2, Qt.SolidLine))
                painter.drawPath(path)
                self.draw_arrowhead(painter, control_point, end)  # Draw arrowhead

            if arrow_type != "feedback loop":
                painter.setPen(pen)
                painter.drawLine(start, end)
                self.draw_arrowhead(painter, start, end)  # Draw arrowhead

    def draw_arrowhead(self, painter, start, end):
        arrow_size = 10
        angle = math.atan2(end.y() - start.y(), end.x() - start.x())

        arrow_p1 = end - QPointF(arrow_size * math.cos(angle - math.pi / 6),
                                 arrow_size * math.sin(angle - math.pi / 6))
        arrow_p2 = end - QPointF(arrow_size * math.cos(angle + math.pi / 6),
                                 arrow_size * math.sin(angle + math.pi / 6))

        arrow_head = QPolygonF([end, arrow_p1, arrow_p2])

        painter.setBrush(Qt.black)
        painter.drawPolygon(arrow_head)

    def is_near(self, pos1, pos2, threshold=10):
        return (pos1 - pos2).manhattanLength() < threshold

    def draw_text_in_middle(self, painter, start, end, text):
        # Draw text in the middle of the line
        midpoint = (start + end) / 2
        painter.setPen(QPen(Qt.black))
        painter.drawText(midpoint, text)

    def hide_all_arrows(self):
        # Hide all arrows by clearing connections
        self.connections = []
        self.viewport().update()

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = Window()
    sys.exit(app.exec_())


SystemExit: 0

C:\Users\Ndivheni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.set StyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_()) ```python
    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
 super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_()) ```python
    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
 super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 626)

In [5]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
 self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 409)

In [6]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouse ReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.set MinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax (2523185284.py, line 135)

In [7]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):  # Fixed method name
        if event.button() == Qt.LeftButton self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax (3940394231.py, line 136)

In [9]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from _db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):  # Fixed method name
        if event.button() == Qt.LeftButton:  # Added colon here
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

 delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 355)

In [10]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label .get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):  # Fixed method name
        if event.button() == Qt.LeftButton:  # Added colon here
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is _near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        ```python
        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)  # Corrected indentation
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax (930925352.py, line 211)

In [11]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    " feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):  # Fixed method name
        if event.button() == Qt.LeftButton:  # Added colon here
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load _project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)  # Corrected indentation
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax (2220667861.py, line 318)

In [14]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
    self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):  # Fixed method name
        if event.button() == Qt.LeftButton:  # Added colon here
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):  # Fixed function name
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)  # Corrected indentation
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

IndentationError: unexpected indent (783045561.py, line 121)

In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event .button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

C:\Users\Ndivheni\AppData\Local\Temp\ipykernel_18016\1298084360.py:182: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p1 = QPoint(
C:\Users\Ndivheni\AppData\Local\Temp\ipykernel_18016\1298084360.py:186: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  p2 = QPoint(


SystemExit: 0

C:\Users\Ndivheni\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label .get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
         save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)  # Corrected indentation
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

IndentationError: unexpected indent (1081824567.py, line 352)

In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project _id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging _end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout .addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3129339172.py, line 78)

In [3]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event .button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

ModuleNotFoundError: No module named 'mysql'

In [4]:
pip install mysql-connector-python

   ---------------------------------------- 0.0/16.1 MB ? eta -:--:--
   --- ------------------------------------ 1.6/16.1 MB 7.6 MB/s eta 0:00:02
   ------- -------------------------------- 2.9/16.1 MB 7.3 MB/s eta 0:00:02
   --------- ------------------------------ 3.7/16.1 MB 5.9 MB/s eta 0:00:03
   ----------- ---------------------------- 4.5/16.1 MB 5.6 MB/s eta 0:00:03
   ------------- -------------------------- 5.5/16.1 MB 5.3 MB/s eta 0:00:02
   ------------- -------------------------- 5.5/16.1 MB 5.3 MB/s eta 0:00:02
   ----------------- ---------------------- 7.1/16.1 MB 4.8 MB/s eta 0:00:02
   ----------------- ---------------------- 7.1/16.1 MB 4.8 MB/s eta 0:00:02
   -------------------- ------------------- 8.4/16.1 MB 4.4 MB/s eta 0:00:02
   ----------------------- ---------------- 9.4/16.1 MB 4.5 MB/s eta 0:00:02
   -------------------------- ------------- 10.7/16.1 MB 4.7 MB/s eta 0:00:02
   ------------------------------ --------- 12.1/16.1 MB 4.8 MB/s eta 0:00:01
   -

In [ ]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event .button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("background-color: white;")
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

In [2]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project _id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging _end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout .addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3129339172.py, line 78)

In [3]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.set Text(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        p1 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle))
        )
        p2 = QPoint(
            self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle)),
            self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle))
        )
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.add Menu("File ")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit ", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout(central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name']}")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SyntaxError: invalid syntax (4096810757.py, line 107)

In [1]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2 px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        p1 = QPoint(
            int(self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle))),
            int(self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle)))
        )
        p2 = QPoint(
            int(self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle))),
            int(self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle)))
        )
        
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout (central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name'] }")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())

SystemExit: 0

C:\Users\Ndivheni\.jupyter\New folder\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels, arrows):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    # Insert arrows
    for arrow in arrows:
        cursor.execute(
            "INSERT INTO arrows (project_id, start_x, start_y, end_x, end_y, color) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, arrow.start_point.x(), arrow.start_point.y(), arrow.end_point.x(), arrow.end_point.y(), arrow.color)
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

def load_arrows_from_db(project_id):
    db = connect _to_db()
    cursor = db.cursor(dictionary=True)

    cursor.execute("SELECT * FROM arrows WHERE project_id = %s", (project_id,))
    arrows = cursor.fetchall()

    cursor.close()
    db.close()

    return arrows

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("border: 1px solid black;")
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)
        self.pos = [0, 0]  # Store position

    def mouseMoveEvent(self, event):
        if event.buttons() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

    @classmethod
    def from_dict(cls, data, parent):
        label = cls(data['text'], parent)
        label.pos = [data['pos_x'], data['pos_y']]
        label.move(data['pos_x'], data['pos_y'])
        return label

class Arrow:
    def __init__(self, start_point, end_point, color, parent):
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.parent = parent

    def show(self):
        self.parent.update()

    def draw(self, painter):
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

    def paintEvent(self, event):
        painter = QPainter(self)
        for arrow in self.arrows:
            arrow.draw(painter)

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Project Management")
        self.setGeometry(100, 100, 1200, 800)

        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QHBoxLayout(self.central_widget)
        self.left_panel = QListWidget()
        self.right_panel = DropArea()

        self.layout.addWidget(self.left_panel)
        self.layout.addWidget(self.right_panel)

        self.populate_left_panel()
        self.create_menu()

    def populate_left_panel(self):
        self.left_panel.addItems(elements)

    def create_menu(self):
        menu_bar = QMenuBar(self)
        self.setMenuBar(menu_bar)

        file_menu = menu_bar.addMenu("File")
        save_action = QAction("Save Project", self)
        load_action = QAction("Load Project", self)

        save_action.triggered.connect(self.save_project)
        load_action.triggered.connect(self.load_project)

        file_menu.addAction(save_action)
        file_menu.addAction(load_action)

    def save_project(self):
        project_name, _ = QInputDialog.getText(self, "Project Name", "Enter project name:")
        project_location, _ = QInputDialog.getText(self, "Project Location", "Enter project location:")
        labels = [{'text': label.text(), 'pos': [label.pos.x(), label.pos.y()]} for label in self.right_panel.labels]
        arrows = self.right_panel.arrows  # Assuming arrows are stored in a way that can be directly used

        save_project_to_db(project_name, project_location, labels, arrows)

    def load_project(self):
        project_id, _ = QInputDialog.getInt(self, "Load Project", "Enter project ID:")
        project, elements = load_project_from_db(project_id)
        self.right_panel.clear()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        arrows = load_arrows_from_db(project_id)
        for arrow in arrows:
            arrow_obj = Arrow(QPoint(arrow['start_x'], arrow ['start_y']), QPoint(arrow['end_x'], arrow['end_y']), arrow['color'], self.right_panel)
            arrow_obj.show()
            self.right_panel.arrows.append(arrow_obj)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()
    sys.exit(app.exec_())
``` This code provides a complete implementation of a project management application that allows users to save and load projects with elements and arrows. The database schema is designed to store projects, elements, and arrows, while the Python code utilizes PyQt5 for the GUI and MySQL for data storage. 

### Key Features:
- **Database Structure**: The SQL code creates three tables: `projects`, `elements`, and `arrows`.
- **Saving Projects**: The `save_project_to_db` function saves the project name, location, elements, and arrows to the database.
- **Loading Projects**: The `load_project` function retrieves the project details, elements, and arrows from the database and displays them in the GUI.
- **Drag-and-Drop Functionality**: Users can drag elements from the left panel to the right panel, where they can be positioned and saved.
- **Arrow Drawing**: The application supports drawing arrows between elements, which are also saved and loaded.

### Usage:
1. **Run the Application**: Execute the Python script to start the application.
2. **Create a Project**: Drag elements from the left panel to the right panel, position them, and save the project.
3. **Load a Project**: Enter the project ID to load previously saved projects along with their elements and arrows.

Make sure to have the necessary libraries installed (`PyQt5` and `mysql-connector-python`) and a MySQL server running with the appropriate credentials. Adjust the database connection parameters as needed.

SyntaxError: invalid syntax (1691133176.py, line 111)

In [4]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels, arrows):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    # Insert arrows
    for arrow in arrows:
        cursor.execute(
            "INSERT INTO arrows (project_id, start_x, start_y, end_x, end_y, color) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, arrow.start_point.x(), arrow.start_point.y(), arrow.end_point.x(), arrow.end_point.y(), arrow.color)
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

def load_arrows_from_db(project_id):
    db = connect_to_db()  # Corrected line
    cursor = db.cursor(dictionary=True)

    cursor.execute("SELECT * FROM arrows WHERE project_id = %s", (project_id,))
    arrows = cursor.fetchall()

    cursor.close()
    db.close()

    return arrows

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
 super().__init__(text, parent)
        self.setAcceptDrops(True)
        self.setStyleSheet("border: 1px solid black;")
        self.setAutoFillBackground(True)
        self.setAlignment(Qt.AlignCenter)
        self.pos = [0, 0]  # Store position

    def mouseMoveEvent(self, event):
        if event.buttons() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

    @classmethod
    def from_dict(cls, data, parent):
        label = cls(data['text'], parent)
        label.pos = [data['pos_x'], data['pos_y']]
        label.move(data['pos_x'], data['pos_y'])
        return label

class Arrow:
    def __init__(self, start_point, end_point, color, parent):
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.parent = parent

    def show(self):
        self.parent.update()

    def draw(self, painter):
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

    def paintEvent(self, event):
        painter = QPainter(self)
        for arrow in self.arrows:
            arrow.draw(painter)

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Project Management")
        self.setGeometry(100, 100, 1200, 800)

        self.status_bar = QStatusBar()
        self.setStatusBar(self.status_bar)

        self.central_widget = QWidget()
        self.setCentralWidget(self.central_widget)

        self.layout = QHBoxLayout(self.central_widget)
        self.left_panel = QListWidget()
        self.right_panel = DropArea()

        self.layout.addWidget(self.left_panel)
        self.layout.addWidget(self.right_panel)

        self.populate_left_panel()
        self.create_menu()

    def populate_left_panel(self):
        self.left_panel.addItems(elements)

    def create_menu(self):
        menu_bar = QMenuBar(self)
        self.setMenuBar(menu_bar)

        file_menu = menu_bar.addMenu("File")
        save_action = QAction("Save Project", self)
        load_action = QAction("Load Project", self)

        save_action.triggered.connect(self.save_project)
        load_action.triggered.connect(self.load_project)

        file_menu.addAction(save_action)
        file_menu.addAction(load_action)

    def save_project(self):
        project_name, _ = QInputDialog.getText(self, "Project Name", "Enter project name:")
        project_location, _ = QInputDialog.getText(self, "Project Location", "Enter project location:")
        labels = [{'text': label.text(), 'pos': [label.pos.x(), label.pos.y()]} for label in self.right_panel.labels]
        arrows = self.right_panel.arrows  # Assuming arrows are stored in a way that can be directly used

        save_project_to_db(project_name, project_location, labels, arrows)

    def load_project(self):
        project_id, _ = QInputDialog.getInt(self, "Load Project", "Enter project ID:")
        project, elements = load_project_from_db(project_id)
        self.right_panel.clear()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        arrows = load_arrows_from_db(project_id)
        for arrow in arrows:
            arrow_obj = Arrow(QPoint(arrow['start_x'], arrow['start_y']), QPoint(arrow['end_x'], arrow['end_y']), arrow['color'], self.right_panel)
            arrow_obj.show()
            self.right_panel.arrows.append(arrow_obj)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_window = MainWindow()
    main_window.show()
    sys.exit(app.exec_())

IndentationError: unindent does not match any outer indentation level (<string>, line 124)

In [ ]:
import sys
import math
import mysql.connector
from PyQt5.QtWidgets import (
    QApplication, QWidget, QHBoxLayout, QVBoxLayout, QLabel, QSplitter,
    QFrame, QMenuBar, QAction, QMainWindow, QMessageBox,
    QDialog, QLineEdit, QPushButton, QFileDialog, QStatusBar, QColorDialog, QListWidget
)
from PyQt5.QtCore import Qt, QMimeData, QPoint
from PyQt5.QtGui import QDrag, QPainter, QPen, QColor

# Data for the legend (left panel)
elements = [
    "________________________________",
    "***     NODES       ***",
    "________________________________",
    "dam",
    "reservoir",
    "pump station",
    "mine",
    "rainwater harvesting",
    "farm block",
    "stormwater",
    "groundwater harvesting",
    "groundwater source",
    "ecosystem infrastructure",
    "mine dam",
    "water treatment plant",
    "wastewater treatment",
    "junction",
    "subarea",
    "water supply area",
    "settlement",
    "power station",
    "industry",
    "agriculture",
    "________________________________",
    "***       DATA      ***",
    "________________________________",
    "water quality monitoring point",
    "streamflow gauging station",
    "water meter",
    "________________________________",
    "***       CONNECTORS       ***",
    "________________________________",
    "runoff",
    "raw water abstraction",
    "treated water",
    "greywater reuse",
    "wastewater",
    "streamflow",
    "treated wastewater",
    "feedback loop"
]

# Database connection function
def connect_to_db():
    return mysql.connector.connect(
        host="localhost",
        user="root",  # replace with your MySQL username
        password="12345678",  # replace with your MySQL password
        database="project_management"
    )

# Function to save a project
def save_project_to_db(project_name, project_location, labels):
    db = connect_to_db()
    cursor = db.cursor()

    # Insert project
    cursor.execute("INSERT INTO projects (name, location) VALUES (%s, %s)", (project_name, project_location))
    project_id = cursor.lastrowid  # Get the ID of the newly created project

    # Insert elements
    for label in labels:
        cursor.execute(
            "INSERT INTO elements (project_id, text, pos_x, pos_y, capacity, type) VALUES (%s, %s, %s, %s, %s, %s)",
            (project_id, label['text'], label['pos'][0], label['pos'][1], label.get('capacity'), label.get('type'))
        )

    db.commit()
    cursor.close()
    db.close()

# Function to load a project
def load_project_from_db(project_id):
    db = connect_to_db()
    cursor = db.cursor(dictionary=True)

    # Load project
    cursor.execute("SELECT * FROM projects WHERE id = %s", (project_id,))
    project = cursor.fetchone()

    # Load elements
    cursor.execute("SELECT * FROM elements WHERE project_id = %s", (project_id,))
    elements = cursor.fetchall()

    cursor.close()
    db.close()

    return project, elements

# Draggable QLabel for legend items
class DraggableLabel(QLabel):
    def __init__(self, text):
        super().__init__(text)
        self.setText(text)

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            drag = QDrag(self)
            mime_data = QMimeData()
            mime_data.setText(self.text())
            drag.setMimeData(mime_data)
            drag.exec_(Qt.MoveAction)

class MovableLabel(QLabel):
    def __init__(self, text, parent=None):
        super().__init__(text, parent)
        self.setText(text)
        self.setStyleSheet("border: 0.2 px solid black; padding: 0 .5px;")
        self._dragging = False
        self.properties = {}  # Store properties of the element

    def mousePressEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = True
            self._drag_start_position = event.pos()

    def mouseMoveEvent(self, event):
        if self._dragging:
            new_position = self.mapToParent(event.pos() - self._drag_start_position)
            self.move(new_position)

    def mouseReleaseEvent(self, event):
        if event.button() == Qt.LeftButton:
            self._dragging = False

    def update_tooltip(self):
        tooltip_text = ", ".join(f"{key}: {value}" for key, value in self.properties.items())
        self.setToolTip(tooltip_text)

    def to_dict(self):
        return {
            'text': self.text(),
            'pos': self.pos().toTuple(),
            'properties': self.properties
        }

    @classmethod
    def from_dict(cls, data, parent=None):
        label = cls(data['text'], parent)
        label.setProperties(data['properties'])
        label.move(*data['pos'])
        return label

    def setProperties(self, properties):
        self.properties = properties
        self.update_tooltip()

class Arrow(QFrame):
    def __init__(self, start_point, end_point, color, parent=None):
        super().__init__(parent)
        self.start_point = start_point
        self.end_point = end_point
        self.color = color
        self.setFixedSize(800, 600)
        self._dragging_start = False
        self._dragging_end = False

    def paintEvent(self, event):
        painter = QPainter(self)
        pen = QPen(QColor(self.color), 2)
        painter.setPen(pen)
        painter.drawLine(self.start_point, self.end_point)
        self.draw_arrowhead(painter)

    def draw_arrowhead(self, painter):
        arrow_length = 10
        arrow_angle = 30
        angle = math.atan2(self.end_point.y() - self.start_point.y(), self.end_point.x() - self.start_point.x())
        
        p1 = QPoint(
            int(self.end_point.x() - arrow_length * math.cos(angle + math.radians(arrow_angle))),
            int(self.end_point.y() - arrow_length * math.sin(angle + math.radians(arrow_angle)))
        )
        p2 = QPoint(
            int(self.end_point.x() - arrow_length * math.cos(angle - math.radians(arrow_angle))),
            int(self.end_point.y() - arrow_length * math.sin(angle - math.radians(arrow_angle)))
        )
        
        painter.drawLine(self.end_point, p1)
        painter.drawLine(self.end_point, p2)

    def mousePressEvent(self, event):
        if self.is_near_point(event.pos(), self.start_point):
            self._dragging_start = True
        elif self.is_near_point(event.pos(), self.end_point):
            self._dragging_end = True

    def mouseMoveEvent(self, event):
        if self._dragging_start:
            self.start_point = event.pos()
            self.update()
        elif self._dragging_end:
            self.end_point = event.pos()
            self.update()

    def mouseReleaseEvent(self, event):
        self._dragging_start = False
        self._dragging_end = False

    def is_near_point(self, point1, point2, threshold=10):
        return (point1.x() >= point2.x() - threshold and point1.x() <= point2.x() + threshold and
                point1.y() >= point2.y() - threshold and point1.y() <= point2.y() + threshold)

    def set_start_point(self, point):
        self.start_point = point
        self.update()

    def set_end_point(self, point):
        self.end_point = point
        self.update()
class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SWENYA SD-APPLICATION")
        self.setGeometry(100, 100, 1200, 600)

        menubar = self.menuBar()
        file_menu = menubar.addMenu("File")

        new_project_action = QAction("New Project", self)
        new_project_action.triggered.connect(self.new_project)
        file_menu.addAction(new_project_action)

        open_project_action = QAction("Open Project", self)
        open_project_action.triggered.connect(self.open_project)
        file_menu.addAction(open_project_action)

        exit_action = QAction("Exit", self)
        exit_action.triggered.connect(self.close)
        file_menu.addAction(exit_action)

        central_widget = QWidget(self)
        layout = QHBoxLayout (central_widget)

        left_panel = self.create_legend_panel()
        self.right_panel = self.create_drop_panel()

        splitter = QSplitter(Qt.Horizontal)
        splitter.addWidget(left_panel)
        splitter.addWidget(self.right_panel)
        splitter.setSizes([400, 800])

        layout.addWidget(splitter)
        central_widget.setLayout(layout)
        self.setCentralWidget(central_widget)

        self.status_bar = QStatusBar(self)
        self.setStatusBar(self.status_bar)

    def create_legend_panel(self):
        legend_panel = QWidget()
        legend_layout = QVBoxLayout()

        for name in elements:
            label = DraggableLabel(name) if not (name.startswith("***") or name.startswith("___")) else QLabel(name)
            legend_layout.addWidget(label)

        legend_panel.setLayout(legend_layout)
        return legend_panel

    def create_drop_panel(self):
        drop_panel = DropArea(self)
        drop_panel.setStyleSheet("background-color: white;")
        return drop_panel

    def new_project(self):
        dialog = ProjectDialog(self)
        if dialog.exec_():
            project_name, project_location = dialog.get_project_info()
            self.clear_workspace()
            self.add_current_project(project_name)
            self.save_project(project_name, project_location)

    def open_project(self):
        projects = self.list_projects()
        dialog = ProjectSelectionDialog(projects, self)
        if dialog.exec_():
            project_id = dialog.get_selected_project_id()
            if project_id:
                self.load_project(project_id)

    def list_projects(self):
        db = connect_to_db()
        cursor = db.cursor(dictionary=True)
        cursor.execute("SELECT * FROM projects")
        projects = cursor.fetchall()
        cursor.close()
        db.close()
        return projects

    def clear_workspace(self):
        self.right_panel.clear()

    def add_current_project(self, project_name):
        if not hasattr(self, 'current_project_menu'):
            self.current_project_menu = self.menuBar().addMenu("Current Project")

        project_action = QAction(project_name, self)
        project_action.triggered.connect(lambda: self.load_project(1))  # Load project with ID 1
        self.current_project_menu.addAction(project_action)

    def save_project(self, project_name, project_location):
        labels = [label.to_dict() for label in self.right_panel.labels]
        save_project_to_db(project_name, project_location, labels)

    def load_project(self, project_id):
        project, elements = load_project_from_db(project_id)
        self.clear_workspace()

        for element in elements:
            label = MovableLabel.from_dict(element, self.right_panel)
            label.show()
            self.right_panel.labels.append(label)

        self.status_bar.showMessage(f"PROJECT '{project['name']}' IS ACTIVE.")

class ProjectDialog(QDialog):
    def __init__(self, parent=None):
        super().__init__(parent)

        self.setWindowTitle("New Project")
        self.setGeometry(100, 100, 400, 200)

        layout = QVBoxLayout()

        self.project_name_input = QLineEdit(self)
        self.project_name_input.setPlaceholderText("Enter Project Name")
        layout.addWidget(self.project_name_input)

        self.project_location_input = QLineEdit(self)
        self.project_location_input.setPlaceholderText("Enter Project Location")
        layout.addWidget(self.project_location_input)

        browse_button = QPushButton("Browse", self)
        browse_button.clicked.connect(self.browse_location)
        layout.addWidget(browse_button)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.accept)
        button_layout.addWidget(save_button)

        delete_button = QPushButton("Delete", self)
        delete_button.clicked.connect(self.delete_project)
        button_layout.addWidget(delete_button)

        preview_button = QPushButton("Preview", self)
        preview_button.clicked.connect(self.preview_project)
        button_layout.addWidget(preview_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def browse_location(self):
        directory = QFileDialog.getExistingDirectory(self, "Select Project Location")
        if directory:
            self.project_location_input.setText(directory)

    def get_project_info(self):
        return self.project_name_input.text(), self.project_location_input.text()

    def delete_project(self):
        QMessageBox.information(self, "Delete", "Project deleted successfully!")

    def preview_project(self):
        QMessageBox.information(self, "Preview", "Project preview functionality will be here!")

class ProjectSelectionDialog(QDialog):
    def __init__(self, projects, parent=None):
        super().__init__(parent)
        self.setWindowTitle("Select a Project")
        layout = QVBoxLayout()

        self.project_list = QListWidget(self)
        for project in projects:
            self.project_list.addItem(f"{project['id']}: {project['name'] }")
        layout.addWidget(self.project_list)

        select_button = QPushButton("Select", self)
        select_button.clicked.connect(self.accept)
        layout.addWidget(select_button)

        self.setLayout(layout)

    def get_selected_project_id(self):
        selected_item = self.project_list.currentItem()
        if selected_item:
            return int(selected_item.text().split(":")[0])
        return None

class DropArea(QFrame):
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setMinimumSize(800, 600)
        self.labels = []
        self.arrows = []

    def dragEnterEvent(self, event):
        if event.mimeData().hasText():
            event.acceptProposedAction()

    def dropEvent(self, event):
        text = event.mimeData().text()
        label = MovableLabel(text, self)
        label.move(event.pos())
        label.show()
        self.labels.append(label)

        if text in ["runoff", "raw water abstraction", "treated water", "greywater reuse", "wastewater", "streamflow", "treated wastewater", "feedback loop"]:
            color = QColorDialog.getColor()
            if color.isValid():
                arrow = Arrow(event.pos(), event.pos() + QPoint(50, 0), color.name(), self)
                arrow.show()
                self.arrows.append(arrow)

        properties_dialog = ElementPropertiesDialog(label, self)
        properties_dialog.exec_()

    def clear(self):
        for label in self.labels:
            label.deleteLater()
        self.labels.clear()
        for arrow in self.arrows:
            arrow.deleteLater()
        self.arrows.clear()

class ElementPropertiesDialog(QDialog):
    def __init__(self, label, parent=None):
        super().__init__(parent)
        self.label = label
        self.setWindowTitle("Element Properties")

        layout = QVBoxLayout()

        self.capacity_input = QLineEdit(self)
        self.capacity_input.setPlaceholderText("Capacity (m³)")
        layout.addWidget(self.capacity_input)

        self.type_input = QLineEdit(self)
        self.type_input.setPlaceholderText("Description (e.g., earth, concrete)")
        layout.addWidget(self.type_input)

        button_layout = QHBoxLayout()
        save_button = QPushButton("Save", self)
        save_button.clicked.connect(self.save_properties)
        button_layout.addWidget(save_button)

        cancel_button = QPushButton("Cancel", self)
        cancel_button.clicked.connect(self.reject)
        button_layout.addWidget(cancel_button)

        layout.addLayout(button_layout)

        self.setLayout(layout)

    def save_properties(self):
        self.label.properties['capacity'] = self.capacity_input.text()
        self.label.properties['type'] = self.type_input.text()
        self.label.update_tooltip()
        self.accept()

# Main application
if __name__ == "__main__":
    app = QApplication(sys.argv)
    main_win = MainWindow()
    main_win.show()
    sys.exit(app.exec_())